In [1]:
!pip install dash

     |████████████████████████████████| 8.5 MB 4.1 MB/s eta 0:00:01
     |████████████████████████████████| 27.7 MB 1.7 MB/s eta 0:00:01
     |████████████████████████████████| 421 kB 2.7 MB/s eta 0:00:01
  Created wheel for dash-html-components: filename=dash_html_components-2.0.0-py3-none-any.whl size=4069 sha256=037ac76e8cb858777e4c5f25b2b73a3287f48b60df9b478605e3f4d18ce10f81
  Stored in directory: /Users/megangentes/Library/Caches/pip/wheels/ec/6b/81/05aceabd8b27f724e2c96784016287cc1bfbc349ebfda451de
  Created wheel for dash-core-components: filename=dash_core_components-2.0.0-py3-none-any.whl size=3801 sha256=8580196c75aadf118ae63caa660dee84d3dbd1c473ebbaff28b342cd7e7a9af4
  Stored in directory: /Users/megangentes/Library/Caches/pip/wheels/00/f9/c7/1a6437d794ed753ea9bc9079e761d4fc803a1f1f5d3697b9ec
  Created wheel for dash-table: filename=dash_table-5.0.0-py3-none-any.whl size=3896 sha256=fb1aab49d1d436680cf2e193634b0cd95a1f6549017872ebb05d15298f6e63dd
  Stored in directory: /User

In [2]:
!pip install wget

  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9681 sha256=894f52e7617841ada8a2245cc7438704b071eadf182fdbd14285bac732b6f46d
  Stored in directory: /Users/megangentes/Library/Caches/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget


In [3]:
import wget

In [4]:
wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv')

'spacex_launch_dash.csv'

In [7]:
!pip install jupyter_dash

In [9]:
!pip install typing-extensions

In [10]:
# Import required libraries
import pandas as pd
from jupyter_dash import JupyterDash
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = JupyterDash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown',
                                             options=[{'label': 'All Sites', 'value': 'ALL'},
                                                      {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                      {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                                      {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                      {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}],
                                             value='ALL',
                                             placeholder='Select a Launch Site here',
                                             searchable=True),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                                min=0, max=10000, step=1000,
                                                marks={0:'0',
                                                       2500:'2500',
                                                       5000:'5000',
                                                       7500:'7500',
                                                       10000:'10000'},
                                                value=[min_payload, max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    filtered_df = spacex_df
    if entered_site == 'ALL':
        fig = px.pie(spacex_df, values='class', 
        names='Launch Site', 
        title='Total Success Launches By Site')
        return fig
    else:
        filtered_df = spacex_df[spacex_df['Launch Site']==entered_site].groupby(['Launch Site', 'class']).size().reset_index(name='class count')
        fig=px.pie(filtered_df, values='class count',
        names='class', 
        title='Total Success Launches for site {}'.format(entered_site))
        return fig
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'),
               Input(component_id='payload-slider', component_property='value')])
def get_scatter_chart(entered_site,entered_payloads):
    sel_payload_min = entered_payloads[0]
    sel_payload_max = entered_payloads[1]
    filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= sel_payload_min) & (spacex_df['Payload Mass (kg)'] <= sel_payload_max)]
    if entered_site == 'ALL':
        fig = px.scatter(filtered_df, x='Payload Mass (kg)',y='class',
        color='Booster Version Category', 
        title='Correlation between Payload and Success for all Sites')
        return fig
    else:
        filtered_dfs = filtered_df[filtered_df['Launch Site']==entered_site]
        fig=px.scatter(filtered_dfs, x='Payload Mass (kg)',y='class', 
        color='Booster Version Category',
        title=f'Correlation between Payload and Success for site {entered_site}')
        return fig

# Run the app
if __name__ == '__main__':
    app.run_server(mode='inline')

/Users/megangentes/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  """
/Users/megangentes/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  
